In [1]:
import os
import json
import logging
from datetime import datetime

In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/harsh_arma/.config/sagemaker/config.yaml


In [3]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)


In [4]:
def check_aws_configuration():
    try:
        # Attempt to create an AWS S3 client
        s3 = boto3.client('s3')
        
        # List buckets to verify AWS credentials and region
        buckets = s3.list_buckets()
        
        print("AWS Configuration is successful.")
        print("Available S3 Buckets:")
        for bucket in buckets['Buckets']:
            print(f"- {bucket['Name']}")
        
        return True
    except Exception as e:
        print(f"Error: {e}")
        print("AWS Configuration is not successful.")
        return False

# Check AWS configuration
check_aws_configuration()

AWS Configuration is successful.
Available S3 Buckets:
- airbyte-data-bucket
- apple-maps-rank-data
- aws-cloudtrail-logs-173829655778-1bc62e66
- aws-sam-cli-managed-default-samclisourcebucket-ujqrexj3n4l8
- cf-templates-1f8yr71fcychj-us-east-1
- chat-widget-dev-serverlessdeploymentbucket-c2tv25twqqud
- chat-widget-local-serverlessdeploymentbucket-5d5l8p5zoshf
- chrone-app-chat-campaign-jsons
- chrone-biz-keyword-jsons
- chrone-biz-react-website-dev
- chrone-biz-website-react
- chrone-community-data
- chrone-email-campaign-jsons
- chrone-global-assets
- chrone-internal-media-generation-reactjs
- chrone-lambda-layers-zip
- chrone-lead-magnet
- chrone-lead-magnet-dev
- chrone-ml-models
- chrone-public-media
- chrone-sales-rank-extraction-jsons
- chrone-sms-campaign-jsons
- chrone-sp-website
- chrone-sp-website-internal
- chrone-sp-website-media-bank
- chrone-sp-website-pages
- chrone-ui-reactjs
- chrone-website
- chrone-website-edit-application
- chrone.ai
- chrone.app
- chrone.biz
- chr

True

In [5]:
# sess = sagemaker.Session()
sess = sagemaker.Session(boto_session=boto3.Session(region_name='us-east-1'))
region = sess.boto_region_name
print(region)

us-east-1


In [6]:
role_arn = "arn:aws:iam::173829655778:role/sage_make_full_access"

In [24]:
bucket_name = "imagetaggingdemo"
data_path = "img_clf_multilabel_lst"

nclasses = 131
nimgs_train = 926
nepochs = 1000
mini_batch_size = 16

train_instance_type = "ml.g4dn.4xlarge"
job_name_prefix = "img-clf-multilabel"

In [25]:
train_image_uri = sagemaker.image_uris.retrieve(
    framework="image-classification",
    region=region,
    image_scope="training",
    version="latest",
)
print(train_image_uri)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:1


In [26]:
s3_output_path = "s3://{}/{}/{}".format(bucket_name, data_path, "model_output")


In [27]:
clf_estimator = sagemaker.estimator.Estimator(
    image_uri=train_image_uri,
    role=role_arn,
    instance_count=1,
    instance_type=train_instance_type,
    volume_size=100,
    max_run=360000,
    input_mode="File",
    output_path=s3_output_path,
    sagemaker_session=sess,
)

In [28]:
clf_estimator.set_hyperparameters(
    num_classes=nclasses,  # update this
    epochs=nepochs,  # update this
    num_training_samples=nimgs_train,  # update this
    mini_batch_size=mini_batch_size,  # update this
    num_layers=18,
    use_pretrained_model=1,
    image_shape="3,224,224",
    resize=256,
    learning_rate=0.001,
    use_weighted_loss=1,
    augmentation_type="crop_color_transform",
    precision_dtype="float32",
    multi_label=1,
)

In [29]:
s3_train_imgs = "s3://{}/{}/{}".format(bucket_name, data_path, "train_imgs")
s3_valid_imgs = "s3://{}/{}/{}".format(bucket_name, data_path, "valid_imgs")
s3_train_annot = "s3://{}/{}/{}".format(bucket_name, data_path, "train_annots")
s3_valid_annot = "s3://{}/{}/{}".format(bucket_name, data_path, "valid_annots")

train_imgs = sagemaker.inputs.TrainingInput(
    s3_train_imgs,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
valid_imgs = sagemaker.inputs.TrainingInput(
    s3_valid_imgs,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
train_annot = sagemaker.inputs.TrainingInput(
    s3_train_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
valid_annot = sagemaker.inputs.TrainingInput(
    s3_valid_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)

data_channels = {
    "train": train_imgs,
    "validation": valid_imgs,
    "train_lst": train_annot,
    "validation_lst": valid_annot,
}

In [13]:
print(sagemaker.__version__)


2.201.0


In [30]:
timestamp = (
    str(datetime.now().replace(microsecond=0)).replace(" ", "-").replace(":", "-")
)
# job_name = "Demo-Multi-Label"
job_name = "Multi-label-Image-Classification" + "-" + timestamp
print(job_name)

Multi-label-Image-Classification-2023-12-22-11-52-16


In [31]:
clf_estimator.fit(inputs=data_channels, logs=True, job_name=job_name)


INFO:sagemaker:Creating training-job with name: Multi-label-Image-Classification-2023-12-22-11-52-16


2023-12-22 06:22:25 Starting - Starting the training job...
2023-12-22 06:22:42 Starting - Preparing the instances for training.....
2023-12-22 06:23:58 Downloading - Downloading input data...
2023-12-22 06:24:28 Downloading - Downloading the training image.................
2023-12-22 06:28:54 Training - Training image download completed. Training in progress...Docker entrypoint called with argument(s): train
Running default environment configuration script
Nvidia gpu devices, drivers and cuda toolkit versions (only available on hosts with GPU):
Fri Dec 22 06:29:42 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memo

In [27]:
print(sagemaker.__version__)
print(sagemaker.__file__)


2.201.0
/home/harsh_arma/Documents/Coding stuff/Intern__/TASK/Image_tagging/ImageClassification/env/lib/python3.10/site-packages/sagemaker/__init__.py


In [32]:
job_name = "Multi-label-Image-Classification"

In [33]:
infer_instance_type = "ml.t2.medium"
model_name = job_name
endpoint_name = job_name

In [34]:
clf_predictor = clf_estimator.deploy(
    initial_instance_count=1,
    instance_type=infer_instance_type,
    endpoint_name=endpoint_name,
    model_name=model_name,
)

INFO:sagemaker:Creating model with name: Multi-label-Image-Classification
INFO:sagemaker:Creating endpoint-config with name Multi-label-Image-Classification
INFO:sagemaker:Creating endpoint with name Multi-label-Image-Classification


---------------------!

In [35]:
sgmkr_runt =  boto3.client("sagemaker-runtime", region_name='us-east-1')


In [36]:
with open("test.jpeg", "rb") as image:
        payload = image.read()
        payload = bytearray(payload)
        
# print(payload)        
response = sgmkr_runt.invoke_endpoint(
    EndpointName = endpoint_name,
    ContentType = 'image/jpeg',
    # Accept = "application/json;verbose",
    Body = payload,
)

prediction = json.loads(response['Body'].read().decode())
print(prediction)

[0.4840906858444214, 0.1124744713306427, 0.4776036739349365, 0.39253881573677063, 0.7884258031845093, 0.4976310133934021, 0.5461063981056213, 0.46138644218444824, 0.4909091889858246, 0.5691899657249451, 0.436217337846756, 0.4376717209815979, 0.4292392134666443, 0.47236600518226624, 0.4703216552734375, 0.44450727105140686, 0.5106387138366699, 0.4922259449958801, 0.11818952113389969, 0.5572019219398499, 0.48253607749938965, 0.4634557366371155, 0.3740185499191284, 0.477821409702301, 0.46375805139541626, 0.3980003893375397, 0.48457637429237366, 0.4510596990585327, 0.5068239569664001, 0.10812534391880035, 0.5296087265014648, 0.46133390069007874, 0.4568847119808197, 0.4560738801956177, 0.48695138096809387, 0.5324930548667908, 0.44196373224258423, 0.5464568734169006, 0.1180095225572586, 0.42284470796585083, 0.1228402629494667, 0.5219780802726746, 0.556522011756897, 0.3826209306716919, 0.42589738965034485, 0.4069894552230835, 0.506653904914856, 0.5096848607063293, 0.47845250368118286, 0.492245

In [49]:
a = prediction
b = ["anesthesia","Upper Body Wax","restorative dentistry","beauty training","Female","Kids","nail services","cosmetic injectables","mens wax","Square","mens colour","Extension services","Old","beauty treatment","portrait photography","grooming","business photography","wispy","Barber","plasma fibrolast","medical services","vitamin injections","wrinkles","microneedling","alternative therapy","wedding services","reflexology","teeth","tatto service","Hair Highlights","Right","body massage","chemical peels","group fitness classes","sugaring","physical therapy","lip treatments","Male","Bohemian Box Braids","piercing services","Locs","body waxing","Landscape","other","dreadlocs","passion twist","Left","cat eye","microblading","none","dermal fillers","sew ins","eyelash services","Gel Nails","lifestyle photography","weight loss services","waxing services","loc services","stretch mark treatment","group services","hair saloons","vajacial","acupunture","Hard Wax","perm","White","energy healing","fitness programs","Braids","Middle","Manicure","cornrows","crystal healing","detoxification services","crochet","eyebrow waxing","Eyebrows","Lower Body Wax","bridal services","Black","Body Contouring","Bridal","Strip Wax","updos","spa","grocery shopping","kids services","bikini wax","hair treatments","wig services","hair styling","artistic services","awakening programs","Dreads","feed ins","makeup services","Eyelash fills","nail polish","facial treatments","beauty services","weaves","Center","Knotless Braids","Brown","cupping therapy","microdermabrasion","personal training","eyebrow services","Pedicure","permanent makeup","skin care services","meditation","iv therepy","Potrait","Eyelash Lift","Acne treatment","yoga and meditation","tanning services","pilates classes","cosmetic dentistry","Box Braids","cooking classes","mens braids","discount services","bohemian twist","barber services","hard","Soft Wax","Facial waxing","light therepy","kinky twist"]
# link a and b
threshold = 0.6  # Set your threshold value here

# Filter elements from list b based on the threshold in list a
filtered_elements = [category for prob, category in zip(a, b) if prob >= threshold]
print([prob for prob, category in zip(a, b) if category == "Extension services"])
# Print the filtered elements
print(filtered_elements)

print(len(a))
print(len(filtered_elements))

[0.4376717209815979]
['Female', 'White', 'Middle', 'Center']
131
4


In [50]:
clf_predictor.delete_endpoint()


INFO:sagemaker:Deleting endpoint configuration with name: Multi-label-Image-Classification
INFO:sagemaker:Deleting endpoint with name: Multi-label-Image-Classification
